In [11]:
import os
import tarfile 
import urllib 

download_root = "https://raw.githubusercontent.com/ageron/handson-ml2/master/" 
housing_path = os.path.join('datasets','housing') 
housing_url = download_root + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url, housing_path):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
        tgz_path = os.path.join(housing_path, "housing.tgz")
        urllib.request.urlretrieve(housing_url, tgz_path)
        housing_tgz = tarfile.open(tgz_path)
        housing_tgz.extractall(path=housing_path)
        housing_tgz.close()

fetch_housing_data(housing_url,housing_path)